In [1]:
from urllib.parse import urlencode
from pyquery import PyQuery as py
import requests
import pandas as pd

In [2]:
#X-Requested-With 用来标识Ajax请求，必须得有
#Referer 用来指明请求来源  必须有
#User-Agent  伪装浏览器，必须有
headers = {'Referer': 'https://m.weibo.cn',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
                'X-Requested-With': 'XMLHttpRequest'}

In [3]:
def get_information(since_id = '',uid = 0,):
    params = {
                'uid': uid,
                'luicode': 10000011,
                'lfid': '231093_-_selffollowed',
                'type': int(uid),
                'value': 6620842908,
                'containerid': '107603' + str(uid),
                'since_id': since_id
    }

    # urlencode() 方法将参数转化为URL的GET请求参数
    url = 'https://m.weibo.cn/api/container/getIndex?' + urlencode(params)
    response = requests.get(url,headers = headers)
    #获得返回的 json 内容，做分析
    json_text = response.json()
    #获得 since_id 为增加页面做准备
    since_id = json_text.get('data').get('cardlistInfo').get('since_id')
    return json_text,since_id

In [4]:
def get_fulltext(mid = ''):
    params = {
                'id': mid
    }
    url = 'https://m.weibo.cn/statuses/extend?' + urlencode(params)
    response = requests.get(url,headers = headers)
    json_text = response.json()
    text = json_text.get('data').get('longTextContent')
    return text

In [5]:
def parse_json(json):
    items = json.get('data').get('cards')


    for item in items:
        item = item.get('mblog')
        mid = item.get('id')
        
        weibo = {}
        weibo['发表时间'] = item.get('created_at')
        weibo['转发数量'] = item.get('reposts_count')
        weibo['点赞数'] = item.get('attitudes_count')
        weibo['评论数'] = item.get('comments_count')
        weibo['内容'] = get_fulltext(mid)
        
        yield weibo

In [9]:

#uid 你所要爬取的微博的ID，在响应的参数列表中可以得到，图中可以找到
uid = 6620842908
#p 爬取的页数
p = 10
data = []
s = get_information(since_id = 4555906027356454,uid = uid)
print(s[1])
for i in parse_json(s[0]):
    print(i)
    data.append(i)
#解析 JSON
#parse_json(s[0])
#输出解析后的内容
'''
for i in parse_json(s[0]):
    print(i)
'''
#多页爬取

for i in range(p):
    s = get_information(since_id = s[1],uid = uid)
    print(s[1])
    for j in parse_json(s[0]):
        print(j)
        data.append(j)

pd.DataFrame(data).to_csv('tnt完整.csv',encoding="utf_8_sig")

4555218308307557
{'发表时间': 'Sat Oct 03 11:35:03 +0800 2020', '转发数量': 11935, '点赞数': 144129, '评论数': 10197, '内容': '<a  href="https://m.weibo.cn/p/index?extparam=%E6%97%B6%E4%BB%A3%E5%B0%91%E5%B9%B4%E5%9B%A2&containerid=10080892b1f367dec8ee89a39109d0b5fe9c0d" data-hide=""><span class=\'url-icon\'><img style=\'width: 1rem;height: 1rem\' src=\'https://n.sinaimg.cn/photo/5213b46e/20180926/timeline_card_small_super_default.png\'></span><span class="surl-text">时代少年团</span></a> 💥<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E5%B0%91%E5%B9%B4OnFire%23&extparam=%23%E5%B0%91%E5%B9%B4OnFire%23" data-hide=""><span class="surl-text">#少年OnFire#</span></a> <a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%97%B6%E4%BB%A3%E5%B0%91%E5%B9%B4%E5%9B%A2%E6%89%93%E6%AD%8C%E8%88%9E%E5%8F%B0%23&extparam=%23%E6%97%B6%E4%BB%A3%E5%B0%91%E5%B9%B4%E5%9B%A2%E6%89%93%E6%AD%8C%E8%88%9E%E5%8F%B0%23" data-hide=""><span class="surl-text">#时代少年团打歌舞台#</span

{'发表时间': 'Fri Oct 02 12:00:04 +0800 2020', '转发数量': 18902, '点赞数': 174892, '评论数': 12277, '内容': '<a  href="https://m.weibo.cn/p/index?extparam=%E6%97%B6%E4%BB%A3%E5%B0%91%E5%B9%B4%E5%9B%A2&containerid=10080892b1f367dec8ee89a39109d0b5fe9c0d" data-hide=""><span class=\'url-icon\'><img style=\'width: 1rem;height: 1rem\' src=\'https://n.sinaimg.cn/photo/5213b46e/20180926/timeline_card_small_super_default.png\'></span><span class="surl-text">时代少年团</span></a>💥<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E5%B0%91%E5%B9%B4OnFire%23&extparam=%23%E5%B0%91%E5%B9%B4OnFire%23" data-hide=""><span class="surl-text">#少年OnFire#</span></a> <br />多种色彩碰撞，无限化学反应<br />10月3日🕙上午10点<br /><br />双人舞台⚡️蓄势待发 '}
{'发表时间': 'Fri Oct 02 11:00:13 +0800 2020', '转发数量': 5578, '点赞数': 105591, '评论数': 5797, '内容': '<a  href="https://m.weibo.cn/p/index?extparam=%E6%97%B6%E4%BB%A3%E5%B0%91%E5%B9%B4%E5%9B%A2&containerid=10080892b1f367dec8ee89a39109d0b5fe9c0d" data-hide=""><span class=\'url-icon\'>

{'发表时间': 'Sat Sep 26 20:20:22 +0800 2020', '转发数量': 151250, '点赞数': 123582, '评论数': 6432, '内容': '<a  href="https://m.weibo.cn/p/index?extparam=%E6%97%B6%E4%BB%A3%E5%B0%91%E5%B9%B4%E5%9B%A2&containerid=10080892b1f367dec8ee89a39109d0b5fe9c0d" data-hide=""><span class=\'url-icon\'><img style=\'width: 1rem;height: 1rem\' src=\'https://n.sinaimg.cn/photo/5213b46e/20180926/timeline_card_small_super_default.png\'></span><span class="surl-text">时代少年团</span></a>💥<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E5%B0%91%E5%B9%B4OnFire%23&extparam=%23%E5%B0%91%E5%B9%B4OnFire%23" data-hide=""><span class="surl-text">#少年OnFire#</span></a> <br />《少年ON FIRE》第一期正片已于各大平台同步上线，感谢合作伙伴<a href=\'/n/新浪综艺\'>@新浪综艺</a> <a href=\'/n/QQ音乐\'>@QQ音乐</a> 的大力支持<span class="url-icon"><img alt=[给你小心心] src="https://h5.sinaimg.cn/m/emoticon/icon/others/qixi2018_xiaoxinxin-c76bf85343.png" style="width:1em; height:1em;" /></span><br />QQ音乐🔗<a data-url="http://t.cn/A64sMK82" href="https://c.y.qq

{'发表时间': 'Wed Sep 23 01:13:09 +0800 2020', '转发数量': 35078, '点赞数': 177659, '评论数': 13600, '内容': '<a  href="https://m.weibo.cn/p/index?extparam=%E6%97%B6%E4%BB%A3%E5%B0%91%E5%B9%B4%E5%9B%A2&containerid=10080892b1f367dec8ee89a39109d0b5fe9c0d" data-hide=""><span class=\'url-icon\'><img style=\'width: 1rem;height: 1rem\' src=\'https://n.sinaimg.cn/photo/5213b46e/20180926/timeline_card_small_super_default.png\'></span><span class="surl-text">时代少年团</span></a>💥<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E5%88%98%E8%80%80%E6%96%870923%E7%94%9F%E6%97%A5%E5%BF%AB%E4%B9%90%23&isnewpage=1" data-hide=""><span class="surl-text">#刘耀文0923生日快乐#</span></a> <br />从小小孩童成为耀眼少年<br />大家见证你成长，也会陪伴你长大<br />希望我们的耀文每一天都开开心心<br />祝<a href=\'/n/时代少年团-刘耀文\'>@时代少年团-刘耀文</a> 15岁生日快乐<span class="url-icon"><img alt=[心] src="https://h5.sinaimg.cn/m/emoticon/icon/others/l_xin-43af9086c0.png" style="width:1em; height:1em;" /></span> '}
{'发表时间': 'Tue Sep 22 21:20:35 +0800 2020', '转发数量': 11

{'发表时间': 'Thu Sep 10 18:00:05 +0800 2020', '转发数量': 20184, '点赞数': 198740, '评论数': 25749, '内容': '<a  href="https://m.weibo.cn/p/index?extparam=%E6%97%B6%E4%BB%A3%E5%B0%91%E5%B9%B4%E5%9B%A2&containerid=10080892b1f367dec8ee89a39109d0b5fe9c0d" data-hide=""><span class=\'url-icon\'><img style=\'width: 1rem;height: 1rem\' src=\'https://n.sinaimg.cn/photo/5213b46e/20180926/timeline_card_small_super_default.png\'></span><span class="surl-text">时代少年团</span></a>💥<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%97%B6%E4%BB%A3%E5%B0%91%E5%B9%B4%E5%9B%A2%E4%BA%8C%E8%BE%91%E3%80%8A%E8%B1%A1%E3%80%8B%23&extparam=%23%E6%97%B6%E4%BB%A3%E5%B0%91%E5%B9%B4%E5%9B%A2%E4%BA%8C%E8%BE%91%E3%80%8A%E8%B1%A1%E3%80%8B%23" data-hide=""><span class="surl-text">#时代少年团二辑《象》#</span></a> <br />【舞·象·之·年】第二篇章《象》<br />-  主打曲《相遇》-<br />-  制作特辑（下）-<br /><br />少年们带大家探秘拍摄地<span class="url-icon"><img alt=[打call] src="https://h5.sinaimg.cn/m/emoticon/icon/default/fb_a1dacall-1e0c4593fc.png" styl

...

{'发表时间': 'Thu Aug 27 15:00:05 +0800 2020', '转发数量': 19064, '点赞数': 129819, '评论数': 13523, '内容': '<a  href="https://m.weibo.cn/p/index?extparam=%E6%97%B6%E4%BB%A3%E5%B0%91%E5%B9%B4%E5%9B%A2&containerid=10080892b1f367dec8ee89a39109d0b5fe9c0d" data-hide=""><span class=\'url-icon\'><img style=\'width: 1rem;height: 1rem\' src=\'https://n.sinaimg.cn/photo/5213b46e/20180926/timeline_card_small_super_default.png\'></span><span class="surl-text">时代少年团</span></a> 💥<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%97%B6%E4%BB%A3%E5%B0%91%E5%B9%B4%E5%9B%A2%E4%BA%8C%E8%BE%91%E3%80%8A%E8%B1%A1%E3%80%8B%23&extparam=%23%E6%97%B6%E4%BB%A3%E5%B0%91%E5%B9%B4%E5%9B%A2%E4%BA%8C%E8%BE%91%E3%80%8A%E8%B1%A1%E3%80%8B%23" data-hide=""><span class="surl-text">#时代少年团二辑《象》#</span></a> <br />【舞·象·之·年】第二篇章《象》<br />-  主打曲《相遇》-<br />-  Single Teaser  -<br /><br />坚定内心<br />驱散耳边呼啸轰鸣<br /><a href=\'/n/时代少年团-严浩翔\'>@时代少年团-严浩翔</a> <br /><br />8月28日10：00<br />《相遇》音源上线 <a data-url="http://t.cn

{'发表时间': 'Tue Aug 25 00:00:43 +0800 2020', '转发数量': 65810, '点赞数': 542491, '评论数': 59097, '内容': '<a  href="https://m.weibo.cn/p/index?extparam=%E6%97%B6%E4%BB%A3%E5%B0%91%E5%B9%B4%E5%9B%A2&containerid=10080892b1f367dec8ee89a39109d0b5fe9c0d" data-hide=""><span class=\'url-icon\'><img style=\'width: 1rem;height: 1rem\' src=\'https://n.sinaimg.cn/photo/5213b46e/20180926/timeline_card_small_super_default.png\'></span><span class="surl-text">时代少年团</span></a>💥<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%97%B6%E4%BB%A3%E5%B0%91%E5%B9%B4%E5%9B%A2%E6%88%90%E5%9B%A2%E4%B8%80%E5%91%A8%E5%B9%B4%23&extparam=%23%E6%97%B6%E4%BB%A3%E5%B0%91%E5%B9%B4%E5%9B%A2%E6%88%90%E5%9B%A2%E4%B8%80%E5%91%A8%E5%B9%B4%23" data-hide=""><span class="surl-text">#时代少年团成团一周年#</span></a> <br />2019——2020<br />我们成为我们的第一年<br />一起看过山川海洋<br />一起有过美梦畅想<br />一起抹去心酸遗憾<br />一起更加闪闪发光<br /><br />我们都会成为更好的自己<br />我们都会成为最好的我们<br />时代少年团，成团一周年快乐！ '}
{'发表时间': 'Mon Aug 24 18:00:03 +0800 2020', '转发数量':

{'发表时间': 'Tue Aug 18 00:03:32 +0800 2020', '转发数量': 20768, '点赞数': 225555, '评论数': 16540, '内容': '<a  href="https://m.weibo.cn/p/index?extparam=%E6%97%B6%E4%BB%A3%E5%B0%91%E5%B9%B4%E5%9B%A2&containerid=10080892b1f367dec8ee89a39109d0b5fe9c0d" data-hide=""><span class=\'url-icon\'><img style=\'width: 1rem;height: 1rem\' src=\'https://n.sinaimg.cn/photo/5213b46e/20180926/timeline_card_small_super_default.png\'></span><span class="surl-text">时代少年团</span></a>💥<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23818%E8%B6%85%E7%BA%A7%E7%A7%80%23&extparam=%23818%E8%B6%85%E7%BA%A7%E7%A7%80%23" data-hide=""><span class="surl-text">#818超级秀#</span></a> <br />以圆满结束过去一天<br />用元气开启全新一天<br />祝大家晚安好梦<span class="url-icon"><img alt=[月亮] src="https://h5.sinaimg.cn/m/emoticon/icon/others/w_yueliang-ec756dd13a.png" style="width:1em; height:1em;" /></span> '}
{'发表时间': 'Mon Aug 17 23:28:43 +0800 2020', '转发数量': 13072, '点赞数': 285802, '评论数': 18410, '内容': '🤗您这边请～请上楼！1️⃣8️⃣'}
{'发表时间': 'Mo

{'发表时间': 'Wed Aug 12 18:00:11 +0800 2020', '转发数量': 47441, '点赞数': 399813, '评论数': 30304, '内容': '<a  href="https://m.weibo.cn/p/index?extparam=%E6%97%B6%E4%BB%A3%E5%B0%91%E5%B9%B4%E5%9B%A2&containerid=10080892b1f367dec8ee89a39109d0b5fe9c0d" data-hide=""><span class=\'url-icon\'><img style=\'width: 1rem;height: 1rem\' src=\'https://n.sinaimg.cn/photo/5213b46e/20180926/timeline_card_small_super_default.png\'></span><span class="surl-text">时代少年团</span></a>💥<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%97%B6%E4%BB%A3%E5%B0%91%E5%B9%B4%E5%9B%A2%E4%BA%8C%E8%BE%91%E3%80%8A%E8%B1%A1%E3%80%8B%23&extparam=%23%E6%97%B6%E4%BB%A3%E5%B0%91%E5%B9%B4%E5%9B%A2%E4%BA%8C%E8%BE%91%E3%80%8A%E8%B1%A1%E3%80%8B%23" data-hide=""><span class="surl-text">#时代少年团二辑《象》#</span></a> <br />【舞·象·之·年】第二篇章《象》<br />-  先行曲《姐姐真漂亮》-<br />-  Teaser 01 -<br /><br />🎧 遮住耳朵关上噪音<br /><span class="url-icon"><img alt=[给你小心心] src="https://h5.sinaimg.cn/m/emoticon/icon/others/qixi2018_xiaoxinxin-c7

{'发表时间': 'Tue Aug 04 18:00:06 +0800 2020', '转发数量': 54041, '点赞数': 342496, '评论数': 48624, '内容': '<a  href="https://m.weibo.cn/p/index?extparam=%E6%97%B6%E4%BB%A3%E5%B0%91%E5%B9%B4%E5%9B%A2&containerid=10080892b1f367dec8ee89a39109d0b5fe9c0d" data-hide=""><span class=\'url-icon\'><img style=\'width: 1rem;height: 1rem\' src=\'https://n.sinaimg.cn/photo/5213b46e/20180926/timeline_card_small_super_default.png\'></span><span class="surl-text">时代少年团</span></a>💥<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E7%88%B1%E8%B1%86v%E5%8A%9B%E9%87%8F%23&extparam=%23%E7%88%B1%E8%B1%86v%E5%8A%9B%E9%87%8F%23" data-hide=""><span class="surl-text">#爱豆v力量#</span></a> <br />TNT《少年•平行的夏》之一 拍摄花絮<br />少年人的夏<br />热血远比午后阳光滚烫<br /><br />擦破皮肤带来细密痛感<br />汗水滴落 蒸发<br />伤疤减淡 消失<br /><br />还好记忆被留下 <a data-url="http://t.cn/A6UV4HBO" href="https://video.weibo.com/show?fid=1034:4534251538808863" data-hide=""><span class=\'url-icon\'><img style=\'width: 1rem;height: 1rem\' src=\'https://h5.s